One-dimensional bound states in quantum mechanics are investigated by using a matrix method to estimate eigenvalues of the Schrodinger operator. Several cases are considered and the answers are compared with theory, including the predictions of perturbation theory and variational methods.

The Schr¨odinger equation in one-dimension (using units where $\hbar = 1 = m$) is
\begin{equation}
    -\frac{1}{2} \frac{d^2\psi_i}{dx^2} + V(x)\psi_i = E_i\psi_i.
\end{equation}
To obtain approximate solutions to this equation, the real-valued position $x$ is replaced by a discrete set of $2N$ points spaced by $\epsilon$, such that $-N\epsilon \leq x < N\epsilon$. The eigenfunction $\psi(x)$, is replaced by a $2N$-dimensional vector $e$, where $\psi(x_n) = e_n$, with $x_n = (n - N)\epsilon$, for $0 \leq n < 2N$.

The Schrodinger equation becomes the matrix eigenvalue equation
\begin{equation}
    Me_i = \epsilon^2E_ie_i,
\end{equation}
where $M$ is a $2N \times 2N$ symmetric tri-diagonal matrix with diagonal entries $c_n = 1 + \epsilon^2V(x_n)$ and off-diagonal entries $b_n = -\frac{1}{2}$ for all $n$.

---

Given a symmetric tri-diagonal matrix $M$, with diagonal entries $c_n$ and off-diagonal entries $b_n$, consider the sequence $q_n$, for $0 \leq n < 2N$, for fixed real parameter $\lambda$,
\begin{align}
    q_0 &= c_0 - \lambda \\
    q_n &= (c_n - \lambda) - b_n^2/q_{n-1}, \quad n > 0.
\end{align}
Let $s(\lambda)$ be the number of the $q_n$ that are negative. Then the number of eigenvalues of $M$ whose values are less than $\lambda$ is $s(\lambda)$. That is, if the eigenvalues are ordered so that $E_i \leq E_{i+1}$, then
\begin{equation}
    \epsilon^2E_i < \lambda \quad \text{for} \quad 0 \leq i < s(\lambda).
\end{equation}
Note that $s(\lambda)$ can be computed as a function of $\lambda$ by starting with a sufficiently small value of $\lambda$, incrementing $\lambda$ in small steps and computing the sequence $\{q_n\}$ for each value. When $s(\lambda)$ increases in value from one step to the next, $\lambda$ must have passed through one eigenvalue of $M$, (or through more than one, if $s(\lambda)$ increases by more than one, in which case we should use smaller stepsize). An accurate value for this eigenvalue can then be determined by bisection before going on to the next eigenvalue.

Once the eigenvalue $E$ has been found sufficiently accurately, to at least $3$ decimal places, the corresponding eigenvector can be found using the equations
\begin{align}
    e_0 &= 1 \\
    e_1 &= 2(c_0 - \epsilon^2E) \\
    e_{n+1} &= 2(c_n - \epsilon^2E)e_n - e_{n-1}, \quad n > 0.
\end{align}
For bound states, the relevant eigenvectors are required to decay exponentially for large $|x|$. It can be shown that the matrix $M$ only has eigenvectors which satisfy this boundary condition.

There are three cautions:
1.  In finding the eigenvalues, there is a division by $q_{n-1}$. Should $q_{n-1}$ become too small it is permissible to replace it by a small default value, to avoid numerical instabilities. The results are unaffected by this procedure.

2.  We compute eigenvectors that are normalised as
\begin{equation}
    \epsilon^2 \sum_{n=0}^{N-1}e_n^2 = 1,
\end{equation}
which corresponds to the physical normalisation
\begin{equation}
    \int_{-\infty}^{\infty} |\psi|^2 \,dx = 1.
\end{equation}
The wavefunction dies away at least exponentially for large $|x|$ so we expect $e_0$ to be very small. For this reason, it is useful to continually normalise the vector $e$, as it is being computed. Specifically, if the $e_n$ have already been calculated for all $n < m$ then it is recommended to normalise them such that
\begin{equation}
    \epsilon \sum_{n=0}^{M-1} e_n^2 = 1
\end{equation}
before computing $e_m$.

3.  The wavefunction also decays exponentially for large positive $x$. This means that for large $n > N$, the values of $e_n$ will become very small. However, if we continue the calculation to very large $n$, numerical errors can lead to exponential growth of the numerical estimates of $e_n$. The calculation is not accurate in this regime: if this happens we will stop the calculation at some $n_{\max} < 2N$, in order to obtain an accurate estimate of the true eigenvector $e$. Alternatively, we can use the fact that all wavefunctions are
either even or odd so the $e_n$ only need to be computed for $0 \leq n \leq N$, where care is needed in normalisation.

In [62]:
import numpy as np
import matplotlib.pyplot as plt

class SchrodingerSolver:
    '''
    A solver for the 1D Schrödinger equation using the matrix method and Given's proceedure.
    '''
    def __init__(self, N, epsilon, potential_func):
        '''
        Initialise the solver with grid parameters and a potential function.
        Args:
            N: Half-number of grid points (total points = 2N).
            epsilon: Grid spacing.
            potential_func: A function V(x) that returns the potential.
        '''
        self.N = N
        self.epsilon = epsilon
        self.num_points = 2 * N

        # Discretise space
        self.x = np.linspace(-N * epsilon, N * epsilon, self.num_points, endpoint=False)

        # Diagonal elements
        self.V = potential_func(self.x)
        self.c = 1.0 + (epsilon**2) * self.V

        # Off-diagonal elements
        self.b_squared = 0.25

    def _count_eigenvalues_below(self, lam):
        '''
        Uses Given's Procedure to count eigenvalues < lambda, i.e., s(lambda).
        '''
        count = 0
        q = self.c[0] - lam

        if q < 0:
            count += 1

        for n in range(1, self.num_points):
            # Handling small q to avoid division by zero
            if abs(q) < 1e-12:
                q = 1e-12
            q = (self.c[n] - lam) - self.b_squared / q

            if q < 0:
                count += 1
        return count

    def find_eigenvalues(self, start_lambda=0.0):
        '''
        Finds the first four eigenvalues starting search from start_lambda.
        Returns a list of Energies (E), not matrix eigenvalues (lambda).
        '''
        energies = []
        current_lambda = start_lambda
        step_size = 0.5 * (self.epsilon**2)

        while len(energies) < 4:
            s_curr = self._count_eigenvalues_below(current_lambda)
            next_lambda = current_lambda + step_size
            s_next = self._count_eigenvalues_below(next_lambda)

            # If count increases, then an eigenvalue is crossed
            if s_next > s_curr:
                # Jumped over a state
                if s_next - s_curr > 1:
                    step_size /= 2
                    continue # Retry this step with smaller size

                # Refine with bisection
                lam_refined = self._bisect(current_lambda, next_lambda, s_curr)

                # Convert matrix lambda to energy E
                E = lam_refined / (self.epsilon**2)
                energies.append(E)

                # Move search forward
                current_lambda = lam_refined + (1e-4 * self.epsilon**2)
            else:
                current_lambda = next_lambda

            # Safety break for infinite loops
            if current_lambda > 100:
                print("Search limit exceeded.")
                break

        return np.array(energies)

    def _bisect(self, lower, upper, target_s_count):
        '''
        Bisection method to refine eigenvalue location.
        '''
        while upper - lower > 1e-12:
            mid = (lower + upper) / 2.0
            if self._count_eigenvalues_below(mid) > target_s_count:
                upper = mid
            else:
                lower = mid
        return (lower + upper) / 2.0

    def compute_eigenvector(self, E, state_index):
        '''
        Computes the eigenvector using symmetry to avoid tail divergence.
        '''
        psi = np.zeros(self.num_points)
        lam = (self.epsilon**2) * E

        # Integrate only from left Boundary (n=0) to center (n=N)
        psi[0] = 1.0
        psi[1] = 2 * (self.c[0] - lam) * psi[0]

        # Stop at the center point x=0
        for n in range(1, self.N):
            psi[n+1] = 2 * (self.c[n] - lam) * psi[n] - psi[n-1]

            # Rescale to prevent overflow during calculation
            if abs(psi[n+1]) > 1e10:
                psi[:n+2] /= np.max(np.abs(psi[:n+2]))

        # Mirror the result to the right side around the center index N
        parity = 1 if (state_index % 2 == 0) else -1
        for k in range(1, self.N):
            if self.N + k < self.num_points:
                psi[self.N + k] = parity * psi[self.N - k]

        # Normalise
        norm = np.sqrt(np.sum(psi**2) * self.epsilon)
        psi = psi / norm

        # Convention: The main lobe on the right side (x > 0) is positive.
        right_half = psi[self.N:]
        # If the main feature on the right is negative, flip the whole wavefunction
        max_val_right = right_half[np.argmax(np.abs(right_half))]
        if max_val_right < 0:
            psi *= -1.0

        return psi

def plot_quantum_states(x, V, energies, wavefunctions):
    '''
    Plots the energy levels and shifted wavefunctions.
    '''
    plt.figure(figsize=(8, 5))
    for i, (E, psi) in enumerate(zip(energies, wavefunctions)):
        plt.plot(x, psi + E, label=f"n={i}, E={E:.3f}")
    plt.title(f"Numeric Bound States")
    plt.xlabel("Position $x$")
    plt.ylabel("Wavefunction $\\psi(x)$")
    plt.ylim(0, np.max(wavefunctions) + np.max(energies))
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

def find_states_and_plot(potential, N_in=50, eps_in=0.1, lam_start=0.0):
    '''
    Finds the first four states and plots them.
    '''
    solutions = SchrodingerSolver(N=N_in, epsilon=eps_in, potential_func=potential)
    energies = solutions.find_eigenvalues(start_lambda=lam_start)
    eigenvectors = [solutions.compute_eigenvector(E, i) for i, E in enumerate(energies)]
    plot_quantum_states(solutions.x, solutions.V, energies, eigenvectors)
    return solutions, energies, eigenvectors